In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

In [5]:
# Set the dataset folder path
dataset_folder = './lol_dataset/'

In [6]:
def load_image(image_path, target_size=(256, 256)):
    """Load and preprocess the image."""
    image = Image.open(image_path)
    image = image.resize(target_size)  # Resize to consistent size
    image = np.array(image, dtype=np.float32) / 255.0  # Normalize to [0, 1] and convert to float32
    return image


# Define folder paths
eval15_low_folder = os.path.join(dataset_folder, 'eval15', 'low')
eval15_high_folder = os.path.join(dataset_folder, 'eval15', 'high')
our485_low_folder = os.path.join(dataset_folder, 'our485', 'low')
our485_high_folder = os.path.join(dataset_folder, 'our485', 'high')

# Get the list of images
eval15_low_images = [os.path.join(eval15_low_folder, img) for img in os.listdir(eval15_low_folder)]
eval15_high_images = [os.path.join(eval15_high_folder, img) for img in os.listdir(eval15_high_folder)]
our485_low_images = [os.path.join(our485_low_folder, img) for img in os.listdir(our485_low_folder)]
our485_high_images = [os.path.join(our485_high_folder, img) for img in os.listdir(our485_high_folder)]

# Load images into arrays
eval15_low_data = [load_image(img_path) for img_path in eval15_low_images]
eval15_high_data = [load_image(img_path) for img_path in eval15_high_images]
our485_low_data = [load_image(img_path) for img_path in our485_low_images]
our485_high_data = [load_image(img_path) for img_path in our485_high_images]

# Combine data from both datasets
X_data = eval15_low_data + our485_low_data
y_data = eval15_high_data + our485_high_data

# Convert to numpy arrays
X_data = np.array(X_data)
y_data = np.array(y_data)

print(f"Low-light data shape: {X_data.shape}")
print(f"High-light data shape: {y_data.shape}")

Low-light data shape: (500, 256, 256, 3)
High-light data shape: (500, 256, 256, 3)


In [7]:
# Split data into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")

Training data shape: (400, 256, 256, 3)
Validation data shape: (100, 256, 256, 3)


In [8]:
def unet_model(input_shape):
    inputs = layers.Input(input_shape)
    
    # Encoder (Downsampling)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)
    
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    # Bottleneck (bottleneck)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)

    # Decoder (Upsampling)
    u3 = layers.UpSampling2D((2, 2))(c4)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u3)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c5)
    
    u2 = layers.UpSampling2D((2, 2))(c5)
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u2)
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c6)

    u1 = layers.UpSampling2D((2, 2))(c6)
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u1)
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c7)
    
    # Output layer
    outputs = layers.Conv2D(3, (1, 1), activation='sigmoid')(c7)

    model = models.Model(inputs, outputs)
    
    return model

# Build the U-Net model
input_shape = (256, 256, 3)
model = unet_model(input_shape)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Summary of the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 256, 256, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 256, 256, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 128, 128, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 128, 128, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 128, 128, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 64, 64, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 64, 64, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 64, 64, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 32, 32, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 32, 32, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 32, 32, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d (UpSampling2D)         │ (None, 64, 64, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 64, 64, 256)         │       1,179,904 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 64, 64, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_1 (UpSampling2D)       │ (None, 128, 128, 256)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 128, 128, 128)       │         295,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 128, 128, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_2 (UpSampling2D)       │ (None, 256, 256, 128)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_12 (Conv2D)                   │ (None, 256, 256, 64)        │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 256, 256, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 7,008,899 (26.74 MB)

 Trainable params: 7,008,899 (26.74 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data generators for low-light and high-light images
train_datagen = ImageDataGenerator(rescale=1./255)  # Normalize pixel values to [0, 1]
val_datagen = ImageDataGenerator(rescale=1./255)

# Set up data generators for training and validation
train_generator = train_datagen.flow(X_train, y_train, batch_size=16)
val_generator = val_datagen.flow(X_val, y_val, batch_size=16)


In [ ]:
# Train the model using the generators
history = model.fit(train_generator, epochs=20, validation_data=val_generator)

# Save the trained U-Net model
model.save('low_light_image_enhanced_unet_model.h5')


Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 372s 15s/step - loss: 0.0500 - val_loss: 0.0464
Epoch 2/20
 3/25 ━━━━━━━━━━━━━━━━━━━━ 6:00 16s/step - loss: 0.0523